In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential #type: ignore
from tensorflow.keras.layers import LSTM, Dense #type: ignore
from datetime import timedelta
import plotly.express as px
from itertools import cycle
import json
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta
import math

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json

def build_predictor(symbol, days_ahead):
    # Download historical stock data for a given symbol from Yahoo Finance and reset the DataFrame index.
    data_frame = yf.download(symbol)
    data_frame.reset_index(inplace=True)
    price_data = data_frame[['Date', 'Close']]

    # Get the current date and set the end date for the data filtering
    current_date = datetime.now().date()
    end_date = current_date + timedelta(days=days_ahead)

    # Filter data to include records between February 15, 2020, and the specified end date
    price_data = price_data[(price_data['Date'] > '2020-02-15') & (price_data['Date'] <= end_date.strftime('%Y-%m-%d'))]
    processed_data = price_data.copy()

    # Scale the closing stock prices to a range between 0 and 1 for normalization.
    final_date = processed_data['Date'].iloc[-1]  # Store the last date for future use.
    processed_data.drop('Date', axis=1, inplace=True)
    price_scaler = MinMaxScaler((0, 1))
    processed_data = price_scaler.fit_transform(np.array(processed_data).reshape(-1, 1))

    # Split the normalized data into 60% training and 40% testing datasets.
    split_ratio = int(len(processed_data) * 0.60)
    train_set, test_set = processed_data[:split_ratio, :], processed_data[split_ratio:, :]

    # Define a function to create input features and target variables arrays for model training/testing.
    def generate_dataset(data, steps=1):
        X, Y = [], []
        for i in range(len(data) - steps - 1):
            X.append(data[i:(i + steps), 0])  # Sequence of 'steps' observations as input features.
            Y.append(data[i + steps, 0])      # Next observation as the target variable.
        return np.array(X), np.array(Y)

    # Create datasets for training and testing the model.
    steps = 15  # Number of past days the model looks at to make a prediction.
    features_train, target_train = generate_dataset(train_set, steps)
    features_test, target_test = generate_dataset(test_set, steps)

    # Reshape input arrays to fit LSTM model expected input [samples, time steps, features].
    features_train = features_train.reshape(features_train.shape[0], features_train.shape[1], 1)
    features_test = features_test.reshape(features_test.shape[0], features_test.shape[1], 1)

    # Initialize, compile and train the LSTM network model.
    lstm_model = Sequential()
    lstm_model.add(LSTM(10, input_shape=(steps, 1), activation='relu'))
    lstm_model.add(Dense(1))
    lstm_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse', 'mae'])
    lstm_model.fit(features_train, target_train, epochs=200, batch_size=32, verbose=1, validation_data=(features_test, target_test))

    # Use the trained model to make predictions on the training and testing datasets.
    train_predictions = lstm_model.predict(features_train)
    test_predictions = lstm_model.predict(features_test)

    # Convert the predictions back to the original scale using the previously defined MinMaxScaler.
    train_predictions = price_scaler.inverse_transform(train_predictions)
    test_predictions = price_scaler.inverse_transform(test_predictions)
    real_train = price_scaler.inverse_transform(target_train.reshape(-1,1))
    real_test = price_scaler.inverse_transform(target_test.reshape(-1,1))

    # Calculate and print evaluation metrics for the model performance.
    print("Training RMSE:", math.sqrt(mean_squared_error(real_train, train_predictions)))
    print("Training R2 score:", r2_score(real_train, train_predictions))
    print("Testing RMSE:", math.sqrt(mean_squared_error(real_test, test_predictions)))
    print("Testing R2 score:", r2_score(real_test, test_predictions))

    # Prepare input for future price prediction based on the last 'steps' observations from the test set.
    future_input = test_set[-steps:].reshape(1, -1)
    future_temp = list(future_input[0])

    # Predict future stock prices for the specified number of days ahead.
    future_predictions = {}
    for i in range(days_ahead):
        future_input = np.array(future_temp[-steps:]).reshape(1, -1, 1)
        predicted_value = lstm_model.predict(future_input, verbose=0)
        predicted_price = price_scaler.inverse_transform(predicted_value)
        future_temp.extend(predicted_value[0])

        future_date = (final_date + timedelta(days=i + 1)).strftime('%Y-%m-%d')
        future_predictions[future_date] = float(predicted_price[0][0])

    # Format the filename based on the stock symbol.
    filename = 'predictions' + symbol.split('-')[0] + '.json'

    # Save the future predictions to a JSON file for further use.
    with open(filename, 'w') as file:
        json.dump(future_predictions, file, indent=4)

    return future_predictions

In [18]:
build_predictor("BTC-USD", 60)

[*********************100%%**********************]  1 of 1 completed


Epoch 1/200
29/29 [==============================] - 5s 26ms/step - loss: 0.2386 - mse: 0.2386 - mae: 0.3972 - val_loss: 0.1378 - val_mse: 0.1378 - val_mae: 0.3144
Epoch 2/200
29/29 [==============================] - 0s 9ms/step - loss: 0.1145 - mse: 0.1145 - mae: 0.2665 - val_loss: 0.0573 - val_mse: 0.0573 - val_mae: 0.1845
Epoch 3/200
29/29 [==============================] - 0s 10ms/step - loss: 0.0424 - mse: 0.0424 - mae: 0.1652 - val_loss: 0.0143 - val_mse: 0.0143 - val_mae: 0.0846
Epoch 4/200
29/29 [==============================] - 0s 11ms/step - loss: 0.0083 - mse: 0.0083 - mae: 0.0735 - val_loss: 0.0012 - val_mse: 0.0012 - val_mae: 0.0245
Epoch 5/200
29/29 [==============================] - 0s 9ms/step - loss: 0.0023 - mse: 0.0023 - mae: 0.0361 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0206
Epoch 6/200
29/29 [==============================] - 0s 13ms/step - loss: 0.0019 - mse: 0.0019 - mae: 0.0312 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0212
Epoch 7/200
29/29 

{'2024-04-22': 65474.1171875,
 '2024-04-23': 65901.6796875,
 '2024-04-24': 66316.78125,
 '2024-04-25': 66801.015625,
 '2024-04-26': 67340.3046875,
 '2024-04-27': 67905.0,
 '2024-04-28': 68433.8984375,
 '2024-04-29': 68965.8203125,
 '2024-04-30': 69575.828125,
 '2024-05-01': 70220.328125,
 '2024-05-02': 70899.75,
 '2024-05-03': 71563.828125,
 '2024-05-04': 72249.265625,
 '2024-05-05': 72948.7890625,
 '2024-05-06': 73676.7734375,
 '2024-05-07': 74429.328125,
 '2024-05-08': 75208.640625,
 '2024-05-09': 76006.484375,
 '2024-05-10': 76820.6640625,
 '2024-05-11': 77656.5,
 '2024-05-12': 78518.6953125,
 '2024-05-13': 79410.6796875,
 '2024-05-14': 80334.171875,
 '2024-05-15': 81294.34375,
 '2024-05-16': 82298.8046875,
 '2024-05-17': 83349.9765625,
 '2024-05-18': 84451.828125,
 '2024-05-19': 85608.46875,
 '2024-05-20': 86824.421875,
 '2024-05-21': 88104.6875,
 '2024-05-22': 89465.0859375,
 '2024-05-23': 90918.25,
 '2024-05-24': 92478.3203125,
 '2024-05-25': 94154.1875,
 '2024-05-26': 95970.2578